In [90]:
import networkx as nx
import random
import math
import matplotlib.pyplot as plt
import math
import random
import numpy as np
import networkx as nx
from joblib import load
import pickle
from node2vec import Node2Vec
from gensim.models import Word2Vec
from networkx.algorithms import community

In [91]:
def create_Big_Graphs():
    num_nodes = 2000
    BigList_2000 =[]
    random_complete = nx.complete_graph(2000)
    for i in range(2000):
        for j in range(i + 1, 2000):
            if random.random() < 0.5:
                random_complete.remove_edge(i, j)
    BigList_2000.append(random_complete)
    #-------------------------------------------------
    random_multigraph = nx.MultiGraph()
    for i in range(num_nodes):
        random_multigraph.add_node(i)
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if random.random() < 0.5:
                random_multigraph.add_edge(i, j)
    BigList_2000.append(random_multigraph)
    #-------------------------------------------------
    pseudograph = nx.MultiGraph()
    pseudograph.add_nodes_from(range(num_nodes))
    for node in pseudograph.nodes():
        pseudograph.add_edge(node, node)
        random_nodes = random.sample(pseudograph.nodes(), k=3)
        pseudograph.add_edges_from([(node, random_node) for random_node in random_nodes])
    BigList_2000.append(pseudograph)
    #-------------------------------------------------
    planar_graph = nx.random_geometric_graph(num_nodes, radius=0.2)
    while not nx.is_connected(planar_graph):
        planar_graph = nx.random_geometric_graph(num_nodes, radius=0.2)
    BigList_2000.append(planar_graph)
    #-------------------------------------------------
    def random_hamiltonian_graph(n):
        graph = nx.random_regular_graph(2, n)
        for i in range(n):
            for j in range(i + 1, n):
                if random.random() < 0.5 and not graph.has_edge(i, j):
                    graph.add_edge(i, j)
        return graph
    random_hamiltonian_graph = random_hamiltonian_graph(num_nodes)
    BigList_2000.append(random_hamiltonian_graph)
    return BigList_2000

In [92]:
def create_main_graphs(num_nodes):
    main_list =[]
   
    random_regular_3 = nx.random_regular_graph(3, num_nodes)
    random_regular_20 = nx.random_regular_graph(20, num_nodes)
    main_list.append(random_regular_3)
    main_list.append(random_regular_20)
    #-------------------------------------------------
    # Create a random connected graph by generating a random spanning tree
    random_connected_tree = nx.random_tree(num_nodes)
    # Add additional edges to maintain connectivity
    while not nx.is_connected(random_connected_tree):
        node1 = random.choice(list(random_connected_tree.nodes()))
        node2 = random.choice(list(random_connected_tree.nodes()))
        if node1 != node2 and not random_connected_tree.has_edge(node1, node2):
            random_connected_tree.add_edge(node1, node2)
    main_list.append(random_connected_tree)
    #-------------------------------------------------
    def generate_random_connected_cycle_graph(num_nodes):
        if num_nodes < 3:
            raise ValueError("Number of nodes must be at least 3 for a cycle.")
        cycle_graph = nx.cycle_graph(num_nodes)
        # Connect the cycle to form a connected cycle graph
        connected_cycle_graph = nx.connected_watts_strogatz_graph(num_nodes, 2, 0.1)
        return connected_cycle_graph
    random_cycle_graph = generate_random_connected_cycle_graph(num_nodes)
    main_list.append(random_cycle_graph)
    #-------------------------------------------------
    def generate_random_connected_barabasi_albert_graph(num_nodes, m):
        if num_nodes <= m:
            raise ValueError("Number of nodes must be greater than m for a Barabási-Albert graph.")
        ba_graph = nx.barabasi_albert_graph(num_nodes, m)
        while not nx.is_connected(ba_graph):
            # Add edges to connect the graph
            non_edges = list(nx.non_edges(ba_graph))
            edge_to_add = non_edges[0]
            ba_graph.add_edge(*edge_to_add)
        return ba_graph
    m_parameter = 2
    random_connected_ba_graph = generate_random_connected_barabasi_albert_graph(num_nodes, m_parameter)
    main_list.append(random_connected_ba_graph)
    #-------------------------------------------------
   
    '''
    def generate_random_connected_erdos_renyi_graph(num_nodes, probability):
        er_graph = nx.erdos_renyi_graph(num_nodes, probability)
        while not nx.is_connected(er_graph):
            # Add edges to connect the graph
            non_edges = list(nx.non_edges(er_graph))
            edge_to_add = non_edges[0]
            er_graph.add_edge(*edge_to_add)
        return er_graph
    edge_probability = 0.1  # Adjust as needed
    random_connected_er_graph = generate_random_connected_erdos_renyi_graph(num_nodes, edge_probability)
    main_list.append(random_connected_er_graph)
    '''
    #-------------------------------------------------
    k = 2  # Each node is connected to k nearest neighbors
    p = 0.3  # Probability of rewiring each edge
    random_graph_ws = nx.watts_strogatz_graph(num_nodes, k, p)
    main_list.append(random_graph_ws)
    #-------------------------------------------------
    k = 10  # Each node is connected to k nearest neighbors
    p = 0.3  # Probability of rewiring each edge
    random_graph_ws = nx.watts_strogatz_graph(num_nodes, k, p)
    main_list.append(random_graph_ws)
    #-------------------------------------------------

    def generate_connected_random_geometric_graph(num_nodes, radius):
        random_geo_graph = nx.random_geometric_graph(num_nodes, radius)
        while not nx.is_connected(random_geo_graph):
            # Connect the graph by adding edges
            non_edges = list(nx.non_edges(random_geo_graph))
            random_edge = random.choice(non_edges)
            random_geo_graph.add_edge(*random_edge)
        return random_geo_graph
    radius = 0.1  # Adjust as needed
    connected_random_geo_graph = generate_connected_random_geometric_graph(num_nodes, radius)
    main_list.append(connected_random_geo_graph)
    #-------------------------------------------------
    def generate_connected_random_internet_graph(num_nodes, k, p):
        internet_graph = nx.random_internet_as_graph(num_nodes)
        while not nx.is_connected(internet_graph):
            non_edges = list(nx.non_edges(internet_graph))
            random_edge = non_edges[0]
            internet_graph.add_edge(*random_edge)
        undirected_internet_graph = internet_graph.to_undirected()
        connected_random_internet_graph = nx.connected_watts_strogatz_graph(num_nodes, k, p)
        return connected_random_internet_graph
    k_parameter = 5
    p_parameter = 0.5
    connected_random_internet_graph = generate_connected_random_internet_graph(num_nodes, k_parameter, p_parameter)
    main_list.append(connected_random_internet_graph)
    #-------------------------------------------------
    return main_list

In [93]:
BigGraphs = create_Big_Graphs()
main_list = create_main_graphs(5000)
main_list2 = create_main_graphs(8000)

In [94]:
from kernighan_lin import kernighan_lin_bisection 

In [95]:
def KernighanLinIterationAndEmbedding(G):
    
    total_vertices = G.number_of_nodes()
    
    # determine total number of iterations
    if total_vertices < 360:
        totalNumberOfIteration = 10
    elif total_vertices > 500:
        totalNumberOfIteration = 0.4 * total_vertices * np.log10(total_vertices)
    else:
        totalNumberOfIteration = total_vertices * np.log10(total_vertices)
    
    totalNumberOfIteration = int(totalNumberOfIteration)
    
    #for j in range(totalNumberOfIteration):
    partition = kernighan_lin_bisection(G, max_iter=totalNumberOfIteration)
    
    G_partition1 = G.subgraph(partition[0])
    G_partition2 = G.subgraph(partition[1])
    
    if nx.is_connected(G_partition1) and nx.is_connected(G_partition2):
        
        partition_1_vertices = G_partition1.number_of_nodes()
        partition_2_vertices = G_partition2.number_of_nodes()
        
        min_vertex_bound = total_vertices/2 - total_vertices*0.01
        max_vertex_bound = total_vertices/2 + total_vertices*0.01
        
        if ((min_vertex_bound <= partition_1_vertices <= max_vertex_bound)
                and (min_vertex_bound <= partition_2_vertices <= max_vertex_bound)):
            return 1
        
        else:
            return 0
    else:
        return 0

In [96]:
# General Libraries
import random as rand
import numpy as np
# Graph Representation & Embedding Library
import networkx as nx 
from node2vec import Node2Vec
import pickle

In [97]:
def getEmbedding(G):
    # Precompute probabilities and generate walks
    node2vec = Node2Vec(G, dimensions=30, walk_length=6, num_walks=10, workers=6)

    # Embed nodes
    model = node2vec.fit(window=5, min_count=1, batch_words=4)

    # Get embeddings for all nodes in the graph
    all_node_embeddings = {node: model.wv[str(node)] for node in G.nodes()}

    return all_node_embeddings
    # loaded_model = Node2Vec.load("node2vec_model.bin")
    # loaded_embedding = loaded_model.wv['0']
    # print(f"Loaded Embedding for Node 0: {loaded_embedding}")

def dictionaryToNpArray(embedding):
    array_from_dict = np.array(list(embedding.values()))
    return array_from_dict

In [98]:
def scriptKullanilacakFonksiyon(G):
    
    # Modeli yükleyip kullanma
    # K-Means modelini yükle
    with open('logistic_regression_model.pickle', 'rb') as model_file:
        loaded_classifier = pickle.load(model_file)

    nodeEmbeddings = getEmbedding(G) 
    nodeEmbeddingsArray = dictionaryToNpArray(nodeEmbeddings)
    graphEmbedding = np.mean(nodeEmbeddingsArray, axis=0).astype('float')

    result_prediction = loaded_classifier.predict(graphEmbedding.reshape(1, -1))
    
    # Prediction 0.5'den buyukse 1 veriyor (Yani partition edilebilir)
    if result_prediction > 0.5:
        return 1
    
    # Prediction 0.5'den kucukse 0 veriyor (Yani partition edilemedi)
    else: 
        return 0



In [99]:
label_val = []

print(len(main_list2))

for graph in main_list2:
    label_val.append(KernighanLinIterationAndEmbedding(graph))

res_val = []

for graph in main_list2:
        res_val.append(scriptKullanilacakFonksiyon(graph))
    
count = 0

for i in range(len(res_val)):
    if res_val[i] == label_val[i]:
        count += 1

print(count)
print(len(res_val))
print(float(count)/len(res_val))
    

9


c:\Users\bora\anaconda3\envs\test1\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.3.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
Computing transition probabilities: 100%|██████████| 8000/8000 [00:00<00:00, 21200.77it/s]
c:\Users\bora\anaconda3\envs\test1\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.3.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
Computing transition probabilities: 100%|██████████| 8000/8000 [00:09<00:00, 876.16it/s]
c:\Users\bora\anaconda3\envs\test1\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.3.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
Computi

PicklingError: Could not pickle the task to send it to the workers.